# ROOTify the Radiological Background Samples

In [1]:
import ROOT
import pandas as pd

In [2]:
fRad = ROOT.TFile("/Users/yuntse/data/lartpc_rd/gampix/radiologicals/gen/fullgeoanatruth-vd1x8x14-1000.root")
fCSV = '/Users/yuntse/data/lartpc_rd/gampix/radiologicals/gen/fullgeoanatruth-vd-reduced_0000.csv'
fROOT = '/Users/yuntse/data/lartpc_rd/gampix/radiologicals/gen/fullgeoanatruth-vd-reduced_0000.root'

## Units

cm, GeV/c, ns

## LAr Geometry

X is the drift direction, vertical \
Xmin = -425.0cm \
Xmax = 425.08cm \
Cathode = -325cm \
CRP = 325cm \
Ymin = -753.8024cm \
Ymax = 753.8024cm \
Zmin = -107.0cm \
Zmax = 2198.88cm

Strategy: To reduce the processing time, I decide to futher reduce the LAr geometry.  I fix the location of neutrino interactions in each sample, i.e. the center in the y-z plane, and cut a square 150cm away from the center of the y-z plane.  Therefore it is a 300x300 cm$^2$ in the y-z plane.  The 150cm is obtained from my containment study for the SNS/COHERENT neutrino source; 96% of the total deposited energy from the SNS $\nu_e$s (from $\pi$ decays at rest) will be contained by the cube border 95cm away from the center (190x190x190cm$^3$).

In [3]:
tRad = fRad.Get("fullgeoanatruth/FullGeoAnaTruth")
tRad.GetListOfBranches().ls()

OBJ: TObjArray	TObjArray	An array of objects : 0
 OBJ: TBranch	run	run/I : 0 at: 0x154aef920
 OBJ: TBranch	subrun	subrun/I : 0 at: 0x154af1700
 OBJ: TBranch	event	event/I : 0 at: 0x154af1960
 OBJ: TBranchElement	label	label : 0 at: 0x154af1bc0
 OBJ: TBranch	pdg_code	pdg_code/I : 0 at: 0x154af8df0
 OBJ: TBranch	x	x/D : 0 at: 0x154af9050
 OBJ: TBranch	y	y/D : 0 at: 0x154af92b0
 OBJ: TBranch	z	z/D : 0 at: 0x154af9510
 OBJ: TBranch	t	t/D : 0 at: 0x154af9770
 OBJ: TBranch	px	px/D : 0 at: 0x154af99d0
 OBJ: TBranch	py	py/D : 0 at: 0x154af9c30
 OBJ: TBranch	pz	pz/D : 0 at: 0x154af9e90
 OBJ: TBranch	Etot	Etot/D : 0 at: 0x154afa0f0
 OBJ: TBranch	mass	mass/D : 0 at: 0x154afa350
 OBJ: TBranch	Ekin	Ekin/D : 0 at: 0x154afa5b0


In [4]:
ymin_cut = -150
ymax_cut = 150
zmin_cut = (-107+2198.88)/2-150
zmax_cut = (-107+2198.88)/2+150

In [5]:
with open(fCSV, 'w') as fOut:
    print('run/I,subrun/I,event/I,pdgCode/I,x/D,y/D,z/D,t/D,px/D,py/D,pz/D,E/D,mass/D', file = fOut)
    for iPart, iEvt in enumerate( tRad ):
        if ( iEvt.y > ymax_cut or iEvt.y < ymin_cut or iEvt.z > zmax_cut or iEvt.z < zmin_cut ): continue
        print(f'{iEvt.run},{iEvt.subrun},{iEvt.event-1},{iEvt.pdg_code},{iEvt.x},{iEvt.y},{iEvt.z},{iEvt.t},{iEvt.px},{iEvt.py},{iEvt.pz},{iEvt.Etot},{iEvt.mass}', file = fOut)

In [7]:
fDest = ROOT.TFile(fROOT, "RECREATE")
newTree = ROOT.TTree("SNEvent", "SNEvent")
newTree.ReadFile(fCSV, "", ",")
newTree.SetDirectory(fDest)
newTree.Write()
fDest.Close()

If you want to have both signals and radiologicals in the same event at this moment (i.e. to merge the signal and radiological files), you need to have the signal csv file ready (use `checkMARLEY.ipynb`) and execute
```shell
sort -t , -k 3,3 -s <signal.csv> <radiological.csv> > SNEvent.csv
```
The order of the signal and radiological files matter.

Then you have to manually remove the headers (two duplicated lines in this case) in the end of the sorted file, and add the header back to the beginning of the file.

In [2]:
fDest = ROOT.TFile("SNEvent.root", "RECREATE")
newTree = ROOT.TTree("SNEvent", "SNEvent")
newTree.ReadFile("SNEvent.csv", "", ",")
newTree.SetDirectory(fDest)
newTree.Write()
fDest.Close()